In [1]:
#Installing necessary libraries:
!pip install -U "transformers>=4.39.0" --q
!pip install --q peft bitsandbytes
!pip install --q accelerate
!pip install -U "trl>=0.8.3" --q
!pip install --q datasets
!pip install --upgrade huggingface_hub
!pip install flash-attn --no-build-isolation

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 44.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.6.1-cp310-cp310-linux_x86_64.whl size=198130327 sha256=2e7a4ae16d20c1c7981f7a9f9a1833b9b528997b990d97c4a6d6d38d3574c773
  Stored in directory: /root/.cache/pip/wheels/91/6a/38/f0faa036b4ac73a73247386f1ab1bb4cb4f6e72e6861a779f1
Successfully built flash-attn


# Fine-tuning Vision Language Model -> LLaVA model on `llava-instruct-mix-vsft`:

In [2]:
! nvidia-smi

Sun Jul 14 05:39:15 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             31W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
#Necesary imports:
import torch
from transformers import AutoTokenizer, AutoProcessor, TrainingArguments, LlavaForConditionalGeneration, BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset

2024-07-14 05:39:23.006237: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 05:39:23.006343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 05:39:23.123428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Loading up the model in 4-bit quantized version:

In [4]:
#Selecting which model to use:
model_card = "bczhou/tiny-llava-v1-hf"

#Configuring the quantization of the model:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,)

#Setting & Loading up the model:
model = LlavaForConditionalGeneration.from_pretrained(
    model_card, 
    quantization_config=quantization_config, 
    torch_dtype=torch.float16
)




config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [5]:
model

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activ

## 2. Creating a base chat template & Initiating LLaVA processor:

In [6]:
BASE_CHAT_TEMPLATE = """
                      A chat between a curious user and an artificial intelligence assistant. \
                      The assistant gives helpful, detailed, and polite answers to the user's questions. \
                      {% for message in messages %}{% if message['role'] == 'user' %}\
                      USER: {% else %}ASSISTANT: {% endif %}{% for item in message['content'] %}{% if item['type'] == 'text' %}{{ item['text'] }}{% elif item['type'] == 'image' %}<image>{% endif %}{% endfor %}\
                      {% if message['role'] == 'user' %} {% else %}{{eos_token}}{% endif %}{% endfor %}"""
                     

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_card)
tokenizer.chat_template = BASE_CHAT_TEMPLATE
processor = AutoProcessor.from_pretrained(model_card)
processor.tokenizer = tokenizer 

tokenizer_config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 3. Loading Up the dataset:

### Creating a DataCollator for processing the data:

In [8]:
class LlavaDataCollator:
    def __init__(self, processor):
        self.processor = processor
        
    def __call__(self, examples):
        texts = []  # -> Creating empty lists to later store the texts seperately
        images = [] # -> Creating an empty list to later store the images seperately
        
        for example in examples:
            messages = example["messages"]
            text = self.processor.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=False)
            texts.append(text)
            
            images.append(example["images"][0])
            
            batch = self.processor(texts, images, return_tensors='pt', padding=True)
            labels = batch["input_ids"].clone()
            if self.processor.tokenizer.pad_token_id is not None:
                labels[labels == self.processor.tokenizer.pad_token_id] = -100
            
            batch["labels"] = labels
            return batch
        
data_collator = LlavaDataCollator(processor)

### Downloading the dataset:

In [9]:
raw_data = load_dataset('HuggingFaceH4/llava-instruct-mix-vsft')
train_data = raw_data["train"]
evaluate_data = raw_data["test"]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/259155 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13640 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [10]:
#Printing the length of training and evaluation dataset:
print("Length of Training Split: ", len(train_data))
print("Length of Evaluation Split: ", len(evaluate_data))

Length of Training Split:  259155
Length of Evaluation Split:  13640


## Fine-Tuning Tiny-LLaVA model(v1.0):

In [11]:
from huggingface_hub import notebook_login

notebook_login()

### a. Setting up the training arguments:

In [17]:
train_args = TrainingArguments(
    output_dir="tiny-llava-v1-finetuned-mix-vsft",
    report_to="tensorboard",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    logging_steps=5,
    num_train_epochs=1,
    max_steps=500,
    push_to_hub=True,
    gradient_checkpointing=True,
    remove_unused_columns=False,
    fp16=True,
    bf16=False
)


### b. Setting up LoRA configuration:

In [18]:
lora_configure = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"]
)

### c. Creating the Supervised Fine-Tuning Trainer(SFTTrainer) object:

In [19]:
trainer = SFTTrainer(
    model=model,
    args=train_args,
    train_dataset=train_data,
    eval_dataset=evaluate_data,
    peft_config=lora_configure,
    dataset_text_field="text", #->dummy field
    tokenizer=tokenizer,
    data_collator=data_collator,
    dataset_kwargs={"skip_prepare_dataset":True}
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:366: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  w

### d. Loading up tensorboard for logging:

In [20]:
%load_ext tensorboard
%tensorboard --logdir /kaggle/working/tiny-llava-v1-finetuned-mix-vsft

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### e. Start the finetuning of the model:

In [21]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
5,2.041700
10,2.182300
15,2.057400
20,1.661400
25,2.038600
30,1.936500
35,1.836100
40,1.952300
45,1.603600
50,1.673400


/opt/conda/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:140: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.42, Please use `text_config.vocab_size` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(


TrainOutput(global_step=500, training_loss=1.1657405271530152, metrics={'train_runtime': 602.4026, 'train_samples_per_second': 6.64, 'train_steps_per_second': 0.83, 'total_flos': 976905881266176.0, 'train_loss': 1.1657405271530152, 'epoch': 0.01543448062972681})

In [22]:
trainer.push_to_hub()

/opt/conda/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:140: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.42, Please use `text_config.vocab_size` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/sid29/tiny-llava-v1-finetuned-mix-vsft/commit/41cb03a700ab2a0dae6e2d75fff8fe04d276eae6', commit_message='End of training', commit_description='', oid='41cb03a700ab2a0dae6e2d75fff8fe04d276eae6', pr_url=None, pr_revision=None, pr_num=None)